Wellcome to my OpenAI Gym Agent

I used the OpenAI Gym to train my agent bot. (https://gym.openai.com)
This python code can be used with any Atari game on openai.com

In [3]:
# Initialization: 

from keras.models import Sequential 
from keras.layers import Dense, Flatten 
from collections import deque

import numpy as np
import gym 
env = gym.make('SpaceInvaders-v0')

import random 

# Create network 
model = Sequential()
model.add(Dense(20, input_shape = (2,) + env.observation_space.shape, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Flatten())
model.add(Dense(18, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(10, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(env.action_space.n, kernel_initializer = 'uniform', activation = 'linear'))
model.compile(loss = 'mse', optimizer = 'adam', metrics = ['accuracy'])

# Parameters 
#where actions will be stored
D = deque()
#Game observing results
observertime = 1000
#Probability of doing a random move
eplison = 0.7
# Discounted Future reward
gama = 0.9
# Learning minibatch size
mb_size = 10

[2017-08-16 12:52:41,630] Making new env: SpaceInvaders-v0


In [4]:
# Knowing what each action does 
observation = env.reset()                     # Game begins
obs = np.expand_dims(observation, axis=0)     # (Formatting issues) Making the observation the first element of a batch of inputs 
state = np.stack((obs, obs), axis=1)
done = False
for t in range(1000):
    if np.random.rand() <= 1:
        action = np.random.randint(0, env.action_space.n, size = 1)[0]
    else:
        Q = model.predict(state)          
        action = np.argmax(Q)             
    observation_new, reward, done, info = env.step(action)     
    obs_new = np.expand_dims(observation_new, axis = 0)          
    state_new = np.append(np.expand_dims(obs_new, axis = 0), state[:, :1, :], axis = 1)     
    D.append((state, action, reward, state_new, done))         
    state = state_new        
    if done:
        env.reset()          
        obs = np.expand_dims(observation, axis = 0)      
        state = np.stack((obs, obs), axis = 1)
print('Observing Finished')

Observing Finished


In [5]:
#observations

minibatch = random.sample(D, mb_size)                              

inputs_shape = (mb_size,) + state.shape[1:]
inputs = np.zeros(inputs_shape)
targets = np.zeros((mb_size, env.action_space.n))

for i in range(0, mb_size):
    state = minibatch[i][0]
    action = minibatch[i][1]
    reward = minibatch[i][2]
    state_new = minibatch[i][3]
    done = minibatch[i][4]
    
# Build equation for the Q function
    inputs[i:i+1] = np.expand_dims(state, axis=0)
    targets[i] = model.predict(state)
    Q_sa = model.predict(state_new)
    
    if done:
        targets[i, action] = reward
    else:
        targets[i, action] = reward + 0.9 * np.max(Q_sa)

# Train network
    model.train_on_batch(inputs, targets)
print('Learning Finished')

Learning Finished


In [6]:
# Play

observation = env.reset()
obs = np.expand_dims(observation, axis=0)
state = np.stack((obs, obs), axis=1)
done = False
tot_reward = 0.0
while not done:
    env.render()                    
    Q = model.predict(state)        
    action = np.argmax(Q)         
    observation, reward, done, info = env.step(action)
    obs = np.expand_dims(observation, axis = 0)
    state = np.append(np.expand_dims(obs, axis = 0), state[:, :1, :], axis = 1)    
    tot_reward += reward
print('Game ended! Total reward: {}'.format(reward))

Game ended! Total reward: 0.0


To gain better results on the output of the game longer training times are required.
Increase the mini batch size in Initialization

![alt text](OutPut.png "Game Results")